# WhatsApp Chat Analysis

In [3]:
# Import modules and libriaries
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import re 
import datetime as dt 

%matplotlib inline

## Importing Text Data

We will now import the text file which contains the whatsapp group chat in read mode using utf-8 encoding.

In [4]:
# import text data
f = open("whatsapp_chat_data.txt", "r", encoding="utf-8")

data = f.read()

In [13]:
whole_text = data.split("\n")
whole_text

['11/14/18, 5:29\u202fAM - +234 803 658 2567: 🤣🤣🤣🤣🤣🤣🤣.... Why I dey laugh sef 🚶🏿🚶🏿🚶🏿',
 '11/14/18, 7:24\u202fAM - +1 (415) 934-8891: https://www.aspenglobalinnovators.org/healthy-communities-fellowship/',
 '4/6/19, 8:15\u202fPM - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.',
 '7/23/19, 10:35\u202fAM - Paul Ashir: Check out MEDICAL BOOKs📚: https://t.me/joinchat/H98arz3ag-TxoE0fx_4Mcw',
 '9/25/19, 1:00\u202fAM - +234 806 965 1788: <Media omitted>',
 '12/6/19, 7:01\u202fAM - Paul Ashir: *HAVE YOU PALPATED YOUR 2020 PRACTICING LICENCE YET?*',
 '',
 'IF NOT:',
 '',
 'GO TO *REMITA.NET* and click on *pay fg and state TSA*.',
 '',
 'Next, CLICK on *FGN:Federal Government of Nigeria*... Fill in the appropriate details and pay. ',
 '',
 '*Login to your mdcn account and upload your CPD points or appointment letter(for residencts)*',
 '',
 "*Then contact one of the persons below (*I contacted Mr Shaub 

In [46]:
# view a random sample of our text data
import random
random.choice(whole_text)

'2/27/23, 12:15\u202fPM - +234 817 950 7218: <Media omitted>'

In [ ]:
\u202fPM

Now lets get a proper structure for our text data. First we will replace `\u202fPM` with an empty sring `""`.

In [47]:
# replace "\u202fPM" with empty string
whole_text_new = []
for text in whole_text:
    # get only text entries with date and time
    if "\u202fPM" in text or "\u202fAM" in text :
        text.replace("\u202fPM", "")
        whole_text_new.append(text)
    else:
        pass

In [48]:
whole_text_new

['11/14/18, 5:29\u202fAM - +234 803 658 2567: 🤣🤣🤣🤣🤣🤣🤣.... Why I dey laugh sef 🚶🏿🚶🏿🚶🏿',
 '11/14/18, 7:24\u202fAM - +1 (415) 934-8891: https://www.aspenglobalinnovators.org/healthy-communities-fellowship/',
 '4/6/19, 8:15\u202fPM - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.',
 '7/23/19, 10:35\u202fAM - Paul Ashir: Check out MEDICAL BOOKs📚: https://t.me/joinchat/H98arz3ag-TxoE0fx_4Mcw',
 '9/25/19, 1:00\u202fAM - +234 806 965 1788: <Media omitted>',
 '12/6/19, 7:01\u202fAM - Paul Ashir: *HAVE YOU PALPATED YOUR 2020 PRACTICING LICENCE YET?*',
 '9/7/20, 11:41\u202fAM - +234 806 921 7904: STEPS TO AVOID BEING HACKED ON WHATSAPP.',
 '10/28/20, 7:08\u202fPM - +234 706 656 2719: @2348037521419  and @2348030619785 . ',
 '11/5/20, 1:17\u202fPM - +234 803 752 1419: Happy birthday sis',
 "11/6/20, 8:45\u202fPM - +234 817 950 7218: Thanks a lot my people. It's much appreciated. 🙏🏾🙏🏾🙏🏾",
 '11/8/20, 6:05\u2

In [88]:
len(whole_text_new)

24056

In [89]:
len(whole_text)

49777

In [68]:
whole_text_new[0]

'11/14/18, 5:29\u202fAM - +234 803 658 2567: 🤣🤣🤣🤣🤣🤣🤣.... Why I dey laugh sef 🚶🏿🚶🏿🚶🏿'

In [91]:
"".join(whole_text_new[0].split("\u202f"))

'11/14/18, 5:29AM - +234 803 658 2567: 🤣🤣🤣🤣🤣🤣🤣.... Why I dey laugh sef 🚶🏿🚶🏿🚶🏿'

In [92]:
whole_text_ = []
for text in whole_text_new:
    whole_text_.append("".join(text.split("\u202f")))

In [93]:
whole_text_ 

['11/14/18, 5:29AM - +234 803 658 2567: 🤣🤣🤣🤣🤣🤣🤣.... Why I dey laugh sef 🚶🏿🚶🏿🚶🏿',
 '11/14/18, 7:24AM - +1 (415) 934-8891: https://www.aspenglobalinnovators.org/healthy-communities-fellowship/',
 '4/6/19, 8:15PM - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.',
 '7/23/19, 10:35AM - Paul Ashir: Check out MEDICAL BOOKs📚: https://t.me/joinchat/H98arz3ag-TxoE0fx_4Mcw',
 '9/25/19, 1:00AM - +234 806 965 1788: <Media omitted>',
 '12/6/19, 7:01AM - Paul Ashir: *HAVE YOU PALPATED YOUR 2020 PRACTICING LICENCE YET?*',
 '9/7/20, 11:41AM - +234 806 921 7904: STEPS TO AVOID BEING HACKED ON WHATSAPP.',
 '10/28/20, 7:08PM - +234 706 656 2719: @2348037521419  and @2348030619785 . ',
 '11/5/20, 1:17PM - +234 803 752 1419: Happy birthday sis',
 "11/6/20, 8:45PM - +234 817 950 7218: Thanks a lot my people. It's much appreciated. 🙏🏾🙏🏾🙏🏾",
 '11/8/20, 6:05AM - +234 803 097 7516: <Media omitted>',
 '11/11/20, 8:59PM - 

In [95]:
xyz = '1/30/22, 12:19PM - +234 803 693 5215: 🤣🤣🤣🤣🤣'
xyz

'1/30/22, 12:19PM - +234 803 693 5215: 🤣🤣🤣🤣🤣'

In [96]:
xyz.split("-")

['1/30/22, 12:19PM ', ' +234 803 693 5215: 🤣🤣🤣🤣🤣']

In [99]:
xyz.split("-")[0]

'1/30/22, 12:19PM '

In [100]:
xyz.split("-")[0][-3:-1]

'PM'

We want to ensure that every text entry has a date and time stamp

Example for dry run : `'16/08/18, 20:09 - '`

\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s

- the first part "d{1,2}" this means here we can have one or 2 digits
- then after we have "/"
- then after we have "d{1,2}" again,same we can have one or 2 digits
- then after we have "/"
- then after we have "d{2,4}" here we check for 2 or more digits
- then we have a space represented as "\s"
- then we have hour representation it can take either one or 2 digits 
- then we have a seperator ":" 

and the pattern repeats so on..

In [ ]:
'1/28/22, 2:14\u202fPM - +234 803 567 4267: Thanks Shittu',

In [62]:
xyz = '1/28/22, 6:40\u202fAM - +234 810 006 7926: Happy birthday Hamdat. More wins and bliss'

pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\.*-\s'

In [67]:
re.split(pattern, xyz)

['1/28/22, 6:40\u202fAM - +234 810 006 7926: Happy birthday Hamdat. More wins and bliss']